<a href="https://colab.research.google.com/github/kimheeseo/pytorch-RNN/blob/main/RNN_%EC%97%B0%EC%8A%B5_2021_12_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
#출처 : https://justkode.kr/deep-learning/pytorch-rnn
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float

"""
Word Processing
"""
word_list = list(set(" ".join(sentences)))
print('word_list값',word_list)


word_list값 ['c', 'd', 'o', 'i', 'g', 'm', ' ', 'y', 'v', 'f', 'k', 'a', 't', 'e', 'u', 'h', 'l']


In [24]:
word_list = list(set(" ".join(sentences).split()))
print(word_list)
word_dict = {w: i for i, w in enumerate(word_list)}
print('word_dict값',word_dict)
number_dict = {i: w for i, w in enumerate(word_list)}
print('number_dict값',number_dict)
n_class = len(word_dict)
print('n_class값',n_class)

['like', 'dog', 'i', 'you', 'coffee', 'cat', 'love', 'hate', 'milk']
word_dict값 {'like': 0, 'dog': 1, 'i': 2, 'you': 3, 'coffee': 4, 'cat': 5, 'love': 6, 'hate': 7, 'milk': 8}
number_dict값 {0: 'like', 1: 'dog', 2: 'i', 3: 'you', 4: 'coffee', 5: 'cat', 6: 'love', 7: 'hate', 8: 'milk'}
n_class값 9


In [55]:
"""
TextRNN Parameter
"""
batch_size = len(sentences)
print('batch_size값',batch_size)
n_step = 2  # 학습 하려고 하는 문장의 길이 - 1
n_hidden = 5  # 은닉층 사이즈
print('sentences값',sentences)

inputtest_batch=[]
targettest_batch=[]
i=0
#다음 for문은 make_batch문의 동작을 보기위해서, 나눠서 표현방법을 본다.
for sen in sentences:
  print(i)
  i=i+1
  word=sen.split()
  print('word값',word)
  input=[word_dict[n] for n in word[:]]
  print('input값',input)
  target=word_dict[word[-1]]
  print('word[-1]값',word[-1])
  #print('np.eye(n_class)값',np.eye(n_class))
  #print('np.eye(n_class)[input]',np.eye(n_class)[input])
  inputtest_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
  #print('inputtest_batch값',inputtest_batch)
  print('target값',target)
  targettest_batch.append(target)

print('inputtest_batch값',inputtest_batch)
print('targettest_batch값',targettest_batch)

#split에 의해 1번째 for문 동작 : 'I like dog'이 sen값이 된다. 그리고 "word=sen.split()"에 의해서
#word값은 ['I','like']가 되고, input값은 [2,0]이 된다. 우선 2개의 값이 나오는 이유는 [:-1]에 의해서 그런것이고,
#만일 [:]이면, ['I','like','dog']가 되고 input값은 [2,0,1]이 되고, 이때 숫자는 앞서 word_dict을 통해 설정한
#I는 2, like는 0, dog는 1을 나타내기 때문에, 이와 같은 결과값이 나온다.

batch_size값 6
sentences값 ['i like dog', 'i love coffee', 'i hate milk', 'you like cat', 'you love milk', 'you hate coffee']
0
word값 ['i', 'like', 'dog']
input값 [2, 0, 1]
word[-1]값 dog
target값 1
1
word값 ['i', 'love', 'coffee']
input값 [2, 6, 4]
word[-1]값 coffee
target값 4
2
word값 ['i', 'hate', 'milk']
input값 [2, 7, 8]
word[-1]값 milk
target값 8
3
word값 ['you', 'like', 'cat']
input값 [3, 0, 5]
word[-1]값 cat
target값 5
4
word값 ['you', 'love', 'milk']
input값 [3, 6, 8]
word[-1]값 milk
target값 8
5
word값 ['you', 'hate', 'coffee']
input값 [3, 7, 4]
word[-1]값 coffee
target값 4
inputtest_batch값 [array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.]]), array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]]), array

In [41]:
def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    word = sen.split()
    input = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]

    input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
    target_batch.append(target)
  
  return input_batch, target_batch  

In [ ]:
input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)

In [ ]:
"""
TextRNN
"""
class TextRNN(nn.Module):
  def __init__(self):
    super(TextRNN, self).__init__()

    self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    self.Softmax = nn.Softmax(dim=1)

  def forward(self, hidden, X):
    X = X.transpose(0, 1)
    outputs, hidden = self.rnn(X, hidden)
    outputs = outputs[-1]  # 최종 예측 Hidden Layer
    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    return model
	

"""
Training
"""
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model(hidden, input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])